In [138]:
import json
from functools import reduce
import itertools

In [6]:
def get_json_data():
    with open('templates/flare.json') as json_data:
        d = json.load(json_data)
    return d

In [26]:
data = get_json_data()

In [124]:
def rename(data, new_name, depth=0, node=[0]):
    
    if node:
        list1 = ['children'] * (len(node))
        list2 = node
    else:
        list1 = ['children'] * (depth + 1)
        list2 = range(depth + 1)
        
        
    dict_path = [x for x in itertools.chain.from_iterable(itertools.izip_longest(list1,list2)) if x!=None]
    reduce(lambda d, k: d[k], dict_path, data)['name'] = new_name
    return data

def view_name(data, depth=0):
    print reduce(lambda d, k: d[k], ['children', 0]* (depth+1), data)['name']

In [135]:
data = rename(data, u'revenue:revenue_equation')
data = rename(data, u'price:10', node = [0,0])
data = rename(data, u'quantity:5', node = [0,1])

In [136]:
data['children'][0]['name']

u'revenue:revenue_equation'

In [137]:
data['children'][0]

{u'children': [{u'children': [{u'name': u'AgglomerativeCluster',
     u'size': 3938},
    {u'name': u'CommunityStructure', u'size': 3812},
    {u'name': u'HierarchicalCluster', u'size': 6714},
    {u'name': u'MergeEdge', u'size': 743}],
   u'name': u'price:10'},
  {u'children': [{u'name': u'BetweennessCentrality', u'size': 3534},
    {u'name': u'LinkDistance', u'size': 5731},
    {u'name': u'MaxFlowMinCut', u'size': 7840},
    {u'name': u'ShortestPaths', u'size': 5914},
    {u'name': u'SpanningTree', u'size': 3416}],
   u'name': u'quantity:5'},
  {u'children': [{u'name': u'AspectRatioBanker', u'size': 7074}],
   u'name': u'optimization'}],
 u'name': u'revenue:revenue_equation'}

In [106]:
view_name(data, 1)

price


In [144]:
#del data['children'][0]['children'][2]

# Store JSON

In [145]:
with open('templates/myjson.json', 'w') as fp:
    json.dump(data, fp)